In [ ]:
import numpy as np
from math import log2
import pandas as pd

In [ ]:
#data generator for classification

In [ ]:
#utils
def split_train_valid_test(data,valid_ratio,test_ratio):
  ```
    TODO: 
      * Shuffle data
      * Split 
              - Train
              - Valid
              - Test
  ```
  shuffled_indcies=np.random.permutation(len(data))
  valid_set_size= int(len(data)*valid_ratio)
  valid_indcies=shuffled_indcies[:valid_set_size]
  test_set_size= int(len(data)*test_ratio)
  test_indcies=shuffled_indcies[valid_set_size:test_set_size+valid_set_size]
  train_indices=shuffled_indcies[test_set_size:]
  return data.iloc[train_indices],data.iloc[valid_indcies],data.iloc[test_indcies]

In [ ]:
class SoftmaxRegression():
  def __init__(self):
    self.W = None
    self.best_W = None
    self.init_w_mean = 0
    self.init_w_std = 1
    self.train_mode = False
    self.train_history = []
    self.valid_history = []

  @staticmethod
  def softmax(Z):
    EZ = np.exp(Z)
    SUM = np.sum(Z, axis=1, keepdims=True)
    return EZ/SUM
  
  @staticmethod
  def cross_entropy(Y_hat, y):
    return -sum([Y_hat[i]*log2(y[i]) for i in range(len(Y_hat))])

  @staticmethod
  def one_hot_encoder(K, batch_label):
    pass
  
  def __init_params(self, number_of_classes, number_of_features):
    self.W = np.random.normal(self.init_w_mean, self.init_w_std, (number_of_classes, number_of_features))
    # add bias = 0
    self.W[:, 0] = 0

  def predict(self, X):
    if self.train_mode:
      W = self.W
    else:
      W = self.best_W
    
    Z = np.dot(X, W.T)
    Y_hat = self.softmax(Z)
    return Y_hat

  def validate(self, X, y):
    Z = np.dot(X, self.W.T)
    Y_hat = self.softmax(Z)
    loss = self.cross_entropy(Y_hat)

    return loss

  def fit(self,
          X_train, y_train,
          X_valid, y_valid,
          eta=0.001,
          BATCH_SIZE=1,
          NUM_EPOCHES=100,
          init_w_mean=0,
          init_w_std=1,
          weight_decay=0.0001,
          weight_decay_type='l2',
          nfreq=500):
    #Initialize
    N, M = X_train.shape
    K = int(np.max(y_train)) + 1
    self.init_w_mean = init_w_mean
    self.init_w_std = init_w_std
    self.__init_params(K, M)

    #Assert for BATCH_SOZE
    if BATCH_SIZE < 1:
      BATCH_SIZE = 1
      print('BATCH_SIZE < 1 => BATCH_SIZE = 1')
    elif BATCH_SIZE > N:
      BATCH_SIZE = N
      print('BATCH_SIZE > N => BATCH_SIZE = N')
    
    #Initialize best_loss
    best_loss = float("inf")
    num_batches = N // BATCH_SIZE #last batch contains all remaining sample
    batch_idx = 0
    train_loss = 0
    self.train_history = []
    self.valid_history = []
    nepoch = 0
    while True:
      if batch_idx > num_batches - 1:
        nepoch += 1
        valid_loss = self.validate(X_valid, y_valid)
        if nepoch%nfreq == 0:
          s_epoch = '{:>7d}'.format(nepoch)
          s_train_loss = '{:>8.5f}'.format(train_loss)
          s_valid_loss = '{:>8.5f}'.format(valid_loss)
          log = 'Epoch: {:s} # Train loss: {:s} # Valid loss: {:s}'.format(s_epoch, s_train_loss, s_valid_loss)
          print(log)

        self.train_history.append(train_loss)
        train_loss = 0
        self.valid_history.append(valid_loss)

        if valid_loss < best_loss:
          best_loss = valid_loss
          self.best_W = self.W.copy()
        
        if nepoch > NUM_EPOCHES:
          break;

        #Decay
        #eta = eta*0.9**(nepoch/20)

        #Shuffle data
        shuffled_idx = np.random.permutation(N)
        X_train = X_train[shuffled_idx, :1]
        y_train = y_train[shuffled_idx]

        #reset
        batch_idx = 0
        start_idx = batch_idx*BATCH_SIZE
        end_idx = min(N, start_idx + BATCH_SIZE)

      elif batch_idx == num_batches - 1:
        start_idx = batch_idx * BATCH_SIZE
        end_idx = N

      else:
        start_idx = batch_index * BATCH_SIZE
        end_indx = min(N, start_idx + BATCH_SIZE)
      
      #Batch data + labels
      batch_data = X_train[start_idx:end_idx, :]
      batch_label = y_train[start_idx:end_idx, :]

      #Forward
      batch_pred = self.predict(batch_data)

      #Loss
      batch_loss = cross_entropy(batch_pred)

      #Regularize
      WW = self.W[:, 1:] #Do not use bias for Regularization
      if weight_decay_type == 'l2':
          l2 = np.sum(WW*WW)
          batch_loss += weight_decay * l2
      else:
        l1 = np.sum(np.abs(WW))
        batch_loss += weight_decay * l1
      
      train_loss += batch_loss

      #Backward pass
      if weight_decay_type == 'l2':
        regu = self.W.copy()
        regu[:, 0] = 0
      else:
        regu = self.W.copy()
        regu[:, 0] = 0
        regu[regu>0] = 1
        regu[regu<0] = -1
        regu[regu==0] = 0
      
      DZ = batch_pred - one_hot_encoder(K, batch_label)
      DW = np.dot(DZ.T, batch_data)
      DW = (1.0/batch_data.shape[0]) * DW + weight_decay * regu

      self.W = self.W - eta * DW
      
      #next batch
      batch_idx += 1
    
    self.train_mode = False

In [ ]:
class LabelTransformer(object):

    def __init__(self, n_classes:int=None):
        self.n_classes = n_classes

    @property
    def n_classes(self):
        return self.__n_classes

    @n_classes.setter
    def n_classes(self, K):
        self.__n_classes = K
        self.__encoder = None if K is None else np.eye(K)

    @property
    def encoder(self):
        return self.__encoder

    def encode(self, class_indices:np.ndarray):
        """
        encode class index into one-of-k code
        Parameters
        ----------
        class_indices : (N,) np.ndarray
            non-negative class index
            elements must be integer in [0, n_classes)
        Returns
        -------
        (N, K) np.ndarray
            one-of-k encoding of input
        """
        if self.n_classes is None:
            self.n_classes = np.max(class_indices) + 1

        return self.encoder[class_indices]

    def decode(self, onehot:np.ndarray):
        return np.argmax(onehot, axis=1)

class SoftmaxRegression(object):
    """
    Softmax regression model
    aka
    multinomial logistic regression,
    multiclass logistic regression,
    maximum entropy classifier.
    y = softmax(X @ W)
    t ~ Categorical(t|y)
    """

    @staticmethod
    def _softmax(a):
        a_max = np.max(a, axis=-1, keepdims=True)
        exp_a = np.exp(a - a_max)
        return exp_a / np.sum(exp_a, axis=-1, keepdims=True)

    def fit(self, X:np.ndarray, t:np.ndarray, max_iter:int=100, learning_rate:float=0.1):
        """
        maximum likelihood estimation of the parameter
        Parameters
        ----------
        X : (N, D) np.ndarray
            training independent variable
        t : (N,) or (N, K) np.ndarray
            training dependent variable
            in class index or one-of-k encoding
        max_iter : int, optional
            maximum number of iteration (the default is 100)
        learning_rate : float, optional
            learning rate of gradient descent (the default is 0.1)
        """
        if t.ndim == 1:
            t = LabelTransformer().encode(t)
        self.n_classes = np.size(t, 1)
        W = np.zeros((np.size(X, 1), self.n_classes))
        for _ in range(max_iter):
            W_prev = np.copy(W)
            y = self._softmax(X @ W)
            grad = X.T @ (y - t)
            W -= learning_rate * grad
            if np.allclose(W, W_prev):
                break
        self.W = W

    def proba(self, X:np.ndarray):
        """
        compute probability of input belonging each class
        Parameters
        ----------
        X : (N, D) np.ndarray
            independent variable
        Returns
        -------
        (N, K) np.ndarray
            probability of each class
        """
        return self._softmax(X @ self.W)

    def classify(self, X:np.ndarray):
        """
        classify input data
        Parameters
        ----------
        X : (N, D) np.ndarray
            independent variable to be classified
        Returns
        -------
        (N,) np.ndarray
            class index for each input
        """
        return np.argmax(self.proba(X), axis=-1)